# CNNs Train

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import time

# Configurações
device = torch.device("cuda:" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Diretórios
train_dir = "..."  # Caminho do conjunto de treino
val_dir = "..."      # Caminho do conjunto de validação
test_dir = "..."    # Caminho do conjunto de teste
img_size = 128
batch_size = 32
epochs = 10
learning_rates = [0.001, 0.0001, 0.00005]  # Testar diferentes taxas de aprendizado

# Transformações
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalização
    ]),
    'val': transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
    ]),
    'test': transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
}

# Datasets e DataLoaders
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_labels = train_dataset.classes
num_classes = len(class_labels)  # Número de classes
print(f"Classes disponíveis: {class_labels}")

# Função para criar diferentes modelos
def create_model(architecture, num_classes):
    if architecture.startswith('resnet'):
        model = getattr(models, architecture)(weights='DEFAULT')
        in_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    elif architecture.startswith('efficientnet'):
        model = getattr(models, architecture)(weights='DEFAULT')
        in_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    elif architecture.startswith('mobilenet'):
        model = getattr(models, architecture)(weights='DEFAULT')
        in_features = model.last_channel
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    elif architecture.startswith('densenet'):
        model = getattr(models, architecture)(weights='DEFAULT')
        in_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    else:
        raise ValueError(f"Arquitetura '{architecture}' não suportada.")
    
    return model

# Função de treinamento com Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs, patience=5):
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    best_val_loss = float('inf')
    best_model_state = None
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        scheduler.step()
        train_loss = running_loss / len(train_loader.dataset)
        history['train_loss'].append(train_loss)

        # Validação
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"Época {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f"Sem melhora em Val Loss por {epochs_no_improve} época(s).")

        if epochs_no_improve >= patience:
            print(f"Early stopping ativado na época {epoch+1}. Melhor Val Loss: {best_val_loss:.4f}")
            break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return history, val_acc

# Testar múltiplas arquiteturas
architectures = ['resnet50', 'efficientnet_b0', 'mobilenet_v2', 'densenet121']
results = {}

for arch in architectures:
    for lr in learning_rates:
        print(f"\nTreinando modelo {arch} com lr={lr}")
        model = create_model(arch, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        
        start_time = time.time()
        history, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs, patience=5)
        elapsed_time = time.time() - start_time
        
        results[(arch, lr)] = (val_acc, elapsed_time)
        torch.save(model.state_dict(), f"model_{arch}_lr{lr}.pth")
        print(f"Modelo {arch} com lr={lr} salvo. Val Acc: {val_acc:.4f}, Tempo: {elapsed_time:.2f}s")

# Melhor modelo
best_model = max(results, key=lambda x: results[x][0])
best_arch, best_lr = best_model
print(f"\nMelhor modelo: {best_arch} com lr={best_lr}, Val Acc: {results[best_model][0]:.4f}")